In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time, gc, pickle
from scipy.stats import skew, kurtosis
# from catboost import CatBoostClassifier
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
# import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import * 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.models import * 
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

DeepCTR version 0.8.1 detected. Your version is 0.7.4.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.1


In [2]:
!pip install deepctr[gpu]==0.7.4
# !pip install tables
# !pip install keras==2.3.1

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 81 kB 689 kB/s eta 0:00:011
     |████████████████████████████████| 320.4 MB 83.0 MB/s eta 0:00:011MB 136 kB/s eta 0:27:06     |██████████████████████▉         | 228.7 MB 952 kB/s eta 0:01:37
     |████████████████████████████████| 6.8 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 96.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 50.4 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 39.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 86.8 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 94.6 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 82.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 92.7 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 107.4 MB/s eta 0:00:01
     |███████████

In [1]:
!pip install pandas==1.1.0

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 10.5 MB 9.3 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [2]:
# train = pd.read_pickle("data/train_fea.pkl")
# test = pd.read_pickle("data/test_fea.pkl") 

# train = pd.read_hdf("data/train_fea.hdf", key='df')
# test = pd.read_hdf("data/test_fea.hdf", key='df')

# print(train.shape, test.shape)

# train = train.sample(30000000)
# train = train[:100000]
# train = train.reset_index(drop=True)
# train.head()
# import pandas as pd 
data = pd.read_pickle('data/train_test_fea.pkl')
data 

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,tags,app_first_class,app_second_class,age,city,city_rank,device_name,device_size,career,gender,net_type,residence,his_app_size,his_on_shelf_time,app_score,emui_dev,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d,communication_hours,uidtask_id_nunique,uidadv_id_nunique,uiddev_id_nunique,uidslot_id_nunique,uidadv_prim_id_nunique,uidspread_app_id_nunique,uidindu_name_nunique,agetask_id_nunique,ageadv_id_nunique,agedev_id_nunique,ageslot_id_nunique,ageadv_prim_id_nunique,agespread_app_id_nunique,ageindu_name_nunique,careertask_id_nunique,careeradv_id_nunique,careerdev_id_nunique,careerslot_id_nunique,careeradv_prim_id_nunique,careerspread_app_id_nunique,careerindu_name_nunique,citytask_id_nunique,cityadv_id_nunique,citydev_id_nunique,cityslot_id_nunique,cityadv_prim_id_nunique,cityspread_app_id_nunique,cityindu_name_nunique,adv_id_count_fea,adv_prim_id_count_fea,task_id_count_fea,uid_count_fea,city_count_fea,cnt_click_of_uid_and_task_id,cnt_click_of_uid_and_adv_id,cnt_click_of_uid_and_spread_app_id,cnt_click_of_uid_and_adv_prim_id,cnt_click_of_uid_and_dev_id,cnt_click_of_uid_and_app_second_class,cnt_click_of_age_and_task_id,cnt_click_of_age_and_adv_id,cnt_click_of_age_and_spread_app_id,cnt_click_of_age_and_adv_prim_id,cnt_click_of_age_and_dev_id,cnt_click_of_age_and_app_second_class,cnt_click_of_gender_and_task_id,cnt_click_of_gender_and_adv_id,cnt_click_of_gender_and_spread_app_id,cnt_click_of_gender_and_adv_prim_id,cnt_click_of_gender_and_dev_id,cnt_click_of_gender_and_app_second_class,cnt_click_of_city_and_task_id,cnt_click_of_city_and_adv_id,cnt_click_of_city_and_spread_app_id,cnt_click_of_city_and_adv_prim_id,cnt_click_of_city_and_dev_id,cnt_click_of_city_and_app_second_class,cnt_click_of_career_and_task_id,cnt_click_of_career_and_adv_id,cnt_click_of_career_and_spread_app_id,cnt_click_of_career_and_adv_prim_id,cnt_click_of_career_and_dev_id,cnt_click_of_career_and_app_second_class,ProNE_Emb_0,ProNE_Emb_1,ProNE_Emb_2,ProNE_Emb_3,ProNE_Emb_4,ProNE_Emb_5,ProNE_Emb_6,ProNE_Emb_7,ProNE_Emb_8,ProNE_Emb_9,ProNE_Emb_10,ProNE_Emb_11,ProNE_Emb_12,ProNE_Emb_13,ProNE_Emb_14,ProNE_Emb_15,ProNE_Emb_16,ProNE_Emb_17,ProNE_Emb_18,ProNE_Emb_19,ProNE_Emb_20,ProNE_Emb_21,ProNE_Emb_22,ProNE_Emb_23,ProNE_Emb_24,ProNE_Emb_25,ProNE_Emb_26,ProNE_Emb_27,ProNE_Emb_28,ProNE_Emb_29,ProNE_Emb_30,ProNE_Emb_31,ProNE_Emb_32,ProNE_Emb_33,ProNE_Emb_34,ProNE_Emb_35,ProNE_Emb_36,ProNE_Emb_37,ProNE_Emb_38,ProNE_Emb_39,ProNE_Emb_40,ProNE_Emb_41,ProNE_Emb_42,ProNE_Emb_43,ProNE_Emb_44,ProNE_Emb_45,ProNE_Emb_46,ProNE_Emb_47
0,0,1924114,2413,2645,7,133,60,5,12,34,13,2,25,7,187,4,65,141,9,2,2,37,-1,-1,1,20,10,2,20,1,-1,2,1^2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^1...,13,24,1,23,1,1,1,1,1,1,1,3777,4089,56,12,107,73,38,4166,4868,56,12,107,74,38,2652,2729,54,12,100,72,36,0.011017,0.200928,0.011017,0.000000,0.160156,0,0,0,0,0,0,0,0,20,14,24,33,0,0,34,20,100,55,0,0,3,4,11,10,1,1,34,23,66,54,-0.266846,0.226318,0.330811,-0.148926,-0.036804,-0.198242,0.192993,-0.221802,0.154175,0.128174,0.259766,-0.260010,0.001972,-0.192871,-0.141357,-0.001098,-0.016495,0.201660,0.015686,-0.065308,-0.028687,0.142456,0.105957,0.217651,0.033997,-0.042419,0.054535,0.077026,0.008865,0.004345,-0.110962,0.021194,-0.089600,0.047729,-0.027878,0.034698,0.011147,0.087769,0.083801,-0.016693,-0.076355,-0.006447,-0.070679,-0.006916,0.027405,0.063171,0.021255,0.439453
1,0,1284962,2336,4517,6,139,60,3,11,78,23,2,23,5,110,5,16,141,4,2,3,20,-1,-1,1,20,4,5,20,-1,-1,2,7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,10,38,1,17,19,20,5,4,10,5,7,4035,4882,56,12,107,73,38,4158,5033,56,12,107,74,38,3353,3794,54,12,103,72,34,0.021622,0.150391,0.021622,0.081604,0.626465,1,1,16,4,14,8,2,2,100,23,100,46,2,2,99,15,100,34,1,1,45,14,54,30,2,2,100,19,100,41,-0.354004,-0.283447,0.326904,-0.126587,0.011856,-0.0

Please check the latest version manually on https://pypi.org/project/deepctr/#history


In [4]:
%%time

train_fea = pd.read_hdf("data/train_1kw_fea.hdf", 'df')
test_fea = pd.read_hdf("data/test_1kw_fea.hdf", 'df') 

uid_emb = pd.read_hdf("data/feature_uid_embed.hdf", 'df')
ua_pagerank = pd.read_hdf("data/feature_uq_pagerank.hdf", 'df')

train_fea = train_fea.merge(uid_emb, on='uid', how='left')
test_fea = test_fea.merge(uid_emb, on='uid', how='left')

train = pd.concat([train_fea, ua_pagerank.iloc[:-1000000]], axis=1)
test = pd.concat([test_fea, ua_pagerank.iloc[-1000000:]], axis=1)

del train_fea, test_fea, uid_emb, ua_pagerank
gc.collect()

DeepCTR version 0.8.0 detected. Your version is 0.7.4.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.0


CPU times: user 36.7 s, sys: 18.8 s, total: 55.5 s
Wall time: 55.5 s


33

In [3]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem,
                                                                          100 * (start_mem - end_mem) / start_mem))
    return df

# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)

# # train = train.sample(frac=1).reset_index(drop=True)
# print(train.shape, test.shape)
# train.head(2)

In [4]:
sparse_features = [
       'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city', 'age', 
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

drop_fea = [col for col in data.columns.tolist() if 'ctr' in col or 'target_enc' in col] # 
print(len(drop_fea))

dense_features = list(set(data.columns) - set(sparse_features + ['label', 'cnt', 'random_sector', 'pt_d', 'id',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
       'uid_app_second_class_ctr'] + drop_fea))
# use_cols = list(set(train.columns) - set(['label']))
print(len(sparse_features), len(dense_features))

0
23 123


In [8]:
test['label'] = -1
data = pd.concat([train, test])
del data['id']
print(data.shape)

del train, test
gc.collect()

(11000000, 168)


0

In [5]:
!free -m
# data[dense_features].count()

              total        used        free      shared  buff/cache   available
Mem:          39978       18051         463          18       21464       21814
Swap:             0           0           0


In [11]:
# data.drop(['uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
#        'uid_app_second_class_ctr'], axis=1, inplace=True) 

In [6]:
# data[sparse_features] = data[sparse_features].fillna(0,)
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

for c in tqdm(dense_features):
#     MAX = data[c].max()
#     MIN = data[c].min()
#     data[c] = (data[c] - MIN) / (MAX - MIN + 1e-8)
    
    mu = np.mean(data[c].values, axis=0)
    sigma = np.std(data[c].values, axis=0) + 1e-8
    data[c] = (data[c] - mu) / sigma

data = reduce_mem_usage(data)

for feat in tqdm(sparse_features):
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# mms = MinMaxScaler(feature_range=(0,1))
# data[dense_features] = mms.fit_transform(data[dense_features])

data = reduce_mem_usage(data)

gc.collect()

  0%|          | 0/23 [00:00<?, ?it/s]

Mem. usage decreased to 2212.52 Mb (59.2% reduction)


100%|██████████| 148/148 [00:23<00:00,  6.42it/s]

Mem. usage decreased to 2159.12 Mb (35.7% reduction)


22

In [7]:
data 

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,tags,app_first_class,app_second_class,age,city,city_rank,device_name,device_size,career,gender,net_type,residence,his_app_size,his_on_shelf_time,app_score,emui_dev,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d,communication_hours,uidtask_id_nunique,uidadv_id_nunique,uiddev_id_nunique,uidslot_id_nunique,uidadv_prim_id_nunique,uidspread_app_id_nunique,uidindu_name_nunique,agetask_id_nunique,ageadv_id_nunique,agedev_id_nunique,ageslot_id_nunique,ageadv_prim_id_nunique,agespread_app_id_nunique,ageindu_name_nunique,careertask_id_nunique,careeradv_id_nunique,careerdev_id_nunique,careerslot_id_nunique,careeradv_prim_id_nunique,careerspread_app_id_nunique,careerindu_name_nunique,citytask_id_nunique,cityadv_id_nunique,citydev_id_nunique,cityslot_id_nunique,cityadv_prim_id_nunique,cityspread_app_id_nunique,cityindu_name_nunique,adv_id_count_fea,adv_prim_id_count_fea,task_id_count_fea,uid_count_fea,city_count_fea,cnt_click_of_uid_and_task_id,cnt_click_of_uid_and_adv_id,cnt_click_of_uid_and_spread_app_id,cnt_click_of_uid_and_adv_prim_id,cnt_click_of_uid_and_dev_id,cnt_click_of_uid_and_app_second_class,cnt_click_of_age_and_task_id,cnt_click_of_age_and_adv_id,cnt_click_of_age_and_spread_app_id,cnt_click_of_age_and_adv_prim_id,cnt_click_of_age_and_dev_id,cnt_click_of_age_and_app_second_class,cnt_click_of_gender_and_task_id,cnt_click_of_gender_and_adv_id,cnt_click_of_gender_and_spread_app_id,cnt_click_of_gender_and_adv_prim_id,cnt_click_of_gender_and_dev_id,cnt_click_of_gender_and_app_second_class,cnt_click_of_city_and_task_id,cnt_click_of_city_and_adv_id,cnt_click_of_city_and_spread_app_id,cnt_click_of_city_and_adv_prim_id,cnt_click_of_city_and_dev_id,cnt_click_of_city_and_app_second_class,cnt_click_of_career_and_task_id,cnt_click_of_career_and_adv_id,cnt_click_of_career_and_spread_app_id,cnt_click_of_career_and_adv_prim_id,cnt_click_of_career_and_dev_id,cnt_click_of_career_and_app_second_class,ProNE_Emb_0,ProNE_Emb_1,ProNE_Emb_2,ProNE_Emb_3,ProNE_Emb_4,ProNE_Emb_5,ProNE_Emb_6,ProNE_Emb_7,ProNE_Emb_8,ProNE_Emb_9,ProNE_Emb_10,ProNE_Emb_11,ProNE_Emb_12,ProNE_Emb_13,ProNE_Emb_14,ProNE_Emb_15,ProNE_Emb_16,ProNE_Emb_17,ProNE_Emb_18,ProNE_Emb_19,ProNE_Emb_20,ProNE_Emb_21,ProNE_Emb_22,ProNE_Emb_23,ProNE_Emb_24,ProNE_Emb_25,ProNE_Emb_26,ProNE_Emb_27,ProNE_Emb_28,ProNE_Emb_29,ProNE_Emb_30,ProNE_Emb_31,ProNE_Emb_32,ProNE_Emb_33,ProNE_Emb_34,ProNE_Emb_35,ProNE_Emb_36,ProNE_Emb_37,ProNE_Emb_38,ProNE_Emb_39,ProNE_Emb_40,ProNE_Emb_41,ProNE_Emb_42,ProNE_Emb_43,ProNE_Emb_44,ProNE_Emb_45,ProNE_Emb_46,ProNE_Emb_47
0,0,0.746582,1226,1434,5,31,44,3,1,22,2,0,14,7,86,0.666504,52,0.161255,7,0,0,26,0.000000,0,0.0,0.529297,0.631348,0.375,1.000000,1,0.0,0,1156,1.000000,13,1,0.956543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.890137,0.749023,1.00000,0.0,1.000000,0.928711,1.000000,1.000000,0.946777,1.000000,0.0,1.000000,1.000000,1.0,0.756348,0.677246,0.959961,1.0,0.949219,0.984863,0.96875,0.011017,0.200928,0.011017,0.000000,0.160156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.199951,0.140015,0.239990,0.330078,0.000000,0.000000,0.340088,0.199951,1.000000,0.549805,0.000000,0.000000,0.029999,0.040009,0.109985,0.099976,0.010002,0.010002,0.340088,0.229980,0.660156,0.540039,0.627441,0.856445,0.583984,0.441162,0.478516,0.400391,0.726074,0.257568,0.593262,0.641602,0.756836,0.202515,0.517090,0.313232,0.408691,0.451172,0.557129,0.629883,0.511230,0.460693,0.412109,0.719727,0.682617,0.723633,0.590332,0.507812,0.526367,0.656738,0.410645,0.509766,0.360352,0.534668,0.413818,0.672363,0.535645,0.449707,0.497803,0.533203,0.676270,0.540527,0.525879,0.460693,0.469238,0.395264,0.488525,0.572266,0.608887,0.731445
1,0,0.230225,1159,3093,4,37,44,1,0,62,11,0,12,5,9,1.000000,4,0.161255,2,0,1,9,0.000000,0,0.0,0.529297,0.315674,0.

# 保存文件

In [20]:
pickle.dump((sparse_features, dense_features), open("all_fea_cols.pkl", 'wb'))

data.to_hdf("data.hdf", 'df')

# 重新读取数据

In [2]:
%%time

# data = pd.read_hdf("data.hdf", 'df')
data = pd.read_pickle("data/train_test.pkl")
# sparse_features, dense_features = pickle.load(open("all_fea_cols.pkl", 'rb'))

CPU times: user 388 ms, sys: 560 ms, total: 948 ms
Wall time: 20.7 s


In [3]:
sparse_features = [
       'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city',
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 'age', 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

drop_fea = [col for col in data.columns.tolist() if 'ctr' in col or 'target_enc' in col] # 
print(len(drop_fea))

dense_features = list(set(data.columns) - set(sparse_features + ['label', 'cnt',
                                                                 'random_sector', 'pt_d', 'id'] + drop_fea))
# use_cols = list(set(train.columns) - set(['label']))
print(len(sparse_features), len(dense_features))

0
24 10


In [4]:
data.shape

(8000000, 36)

In [5]:
dense_features 

['membership_life_duration',
 'device_price',
 'communication_avgonline_30d',
 'city_rank',
 'app_score',
 'list_time',
 'up_life_duration',
 'device_size',
 'emui_dev',
 'his_app_size']

In [8]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=16)
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [9]:
print(data.shape, data['pt_d'].values)

(8000000, 148) [1 1 1 ... 8 8 8]


In [8]:
trn_size = 10000000
train = data.iloc[:trn_size].reset_index(drop=True)
test = data.iloc[trn_size:].reset_index(drop=True)
print(train.shape, test.shape)

del data
gc.collect()

(10000000, 168) (1000000, 168)


22

In [ ]:
# 5折交叉

file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("deep_base")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    # print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")

class Evaluate(Callback):
    def __init__(self, fold, valid_model_input, valid_label):
        self.best_auc = 0.0 
        self.fold = fold
        self.valid_model_input = valid_model_input
        self.valid_label = valid_label
     
    def on_epoch_end(self, epoch, logs=None):
        score = self.evaluate()
        if score > self.best_auc:
            self.best_auc = score
            model.save_weights('data/best_model_for_ctr_{}.weights'.format(self.fold))
        print('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
        write_log('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
        
    def evaluate(self):
        val_pred = model.predict(self.valid_model_input, verbose=1, batch_size=1024)   # [bs, 10]
        return auc(self.valid_label, val_pred)        


train_pred = np.zeros((train.shape[0], 1))
test_pred = np.zeros((test.shape[0], 1))

n_splits = 5
# fold = GroupKFold(n_splits=n_splits)
fold = KFold(n_splits=n_splits, shuffle=True, random_state=2020)
# groups = train_df['loadingOrder'].values
kf_way = fold.split(train)

# model input
test_model_input = {name:test[name].values for name in feature_names}
label = train['label'].values

# train
for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
    
    print("fold ", n_fold)
    
    train_data = train.iloc[train_idx]
    valid_data = train.iloc[valid_idx]
    
    train_model_input = {name: train_data[name].values for name in tqdm(feature_names)}
    valid_model_input = {name: valid_data[name].values for name in tqdm(feature_names)}
    
    model = FiBiNET(linear_feature_columns, dnn_feature_columns, task='binary',
                    dnn_dropout=0.2, dnn_hidden_units=(512, 128),)
    model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'],)

    evaluator = Evaluate(n_fold, valid_model_input, label[valid_idx])
    history = model.fit(train_model_input, label[train_idx],
                    batch_size=2048, epochs=5, verbose=1, callbacks=[evaluator])
    
    model.load_weights("data/best_model_for_ctr_{}.weights".format(n_fold))
    train_pred[valid_idx] = model.predict(valid_model_input, batch_size=1024, verbose=1)
    test_pred += model.predict(test_model_input, batch_size=1024, verbose=1) / n_splits
    
    del model, train_model_input, valid_model_input
    gc.collect()

print(auc(label, train_pred.reshape(-1)))

fold  1



100%|██████████| 132/132 [00:00<00:00, 21415.24it/s]

100%|██████████| 132/132 [00:00<00:00, 22564.73it/s]


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'senet_layer/W_1:0' shape=(23, 7) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'senet_layer/W_1:0' shape=(23, 7) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


Epoch 1/5


...

The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'senet_layer_1/W_1:0' shape=(23, 7) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'senet_layer_1/W_1:0' shape=(23, 7) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


Epoch 1/5


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1954/1954 [==============================] - 42s 21ms/step
current auc: 0.807888, best auc: 0.807888

3907/3907 [==============================] - 417s 107ms/step - loss: 0.1262 - binary_crossentropy: 0.1259
Epoch 2/5
1954/1954 [==============================] - 41s 21ms/step
current auc: 0.811854, best auc: 0.811854

3907/3907 [==============================] - 404s 104ms/step - loss: 0.1235 - binary_crossentropy: 0.1230
Epoch 3/5
 810/3907 [=====>........................] - ETA: 4:45 - loss: 0.1232 - binary_crossentropy: 0.1228

In [12]:
print(auc(label, train_pred.reshape(-1))) 

0.8139674200156203


In [14]:
test_pred 

array([[0.00666512],
       [0.08508157],
       [0.1260884 ],
       ...,
       [0.04818786],
       [0.01417376],
       [0.0217111 ]])

In [15]:
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': test_pred.reshape(-1), 
    })

result.to_csv("submission.csv", index=False) 

In [10]:

train = data[data["pt_d"]<7].reset_index(drop=True)
valid = data[data["pt_d"]==7].reset_index(drop=True)
test = data.iloc[-1000000:].reset_index(drop=True)

print(train.shape, valid.shape, test.shape)

train_model_input = {name:train[name].values for name in feature_names}
valid_model_input = {name:valid[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

train_label = train['label'].values
valid_label = valid['label'].values
print(train_label.shape, valid_label.shape)

del train, valid, test, data
gc.collect()

(6000000, 148) (1000000, 148) (1000000, 148)
(6000000,) (1000000,)


0

In [35]:
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary', 
               dnn_dropout=0.2, dnn_hidden_units=(512, 256))
# model = FiBiNET(linear_feature_columns,dnn_feature_columns, task='binary',
#                 dnn_dropout=0.2, dnn_hidden_units=(512, 128),)
# model = xDeepFM(linear_feature_columns,dnn_feature_columns, task='binary',
#                 dnn_dropout=0.2, dnn_hidden_units=(512, 128),)
# model = FLEN(linear_feature_columns, dnn_feature_columns, task='binary',
#             dnn_dropout=0.2, dnn_hidden_units=(512, 128),)

model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'],)

In [19]:
!free -m
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:          39978       23382         311          18       16285       16484
Swap:             0           0           0
Thu Sep  3 13:03:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:08.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |     10MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                            

In [20]:
# !pip install keras==2.2.4

file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("deep_base")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    # print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")

In [36]:
# from sklearn.metrics import accuracy_score 

class Evaluate(Callback):
    def __init__(self):
        self.best_auc = 0.0 
     
    def on_epoch_end(self, epoch, logs=None):
        score = self.evaluate()
        if score > self.best_auc:
            self.best_auc = score
            model.save_weights('data/best_model_for_ctr.weights')
        print('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
        write_log('current auc: %.6f, best auc: %.6f\n' % (score, self.best_auc))
        
    def evaluate(self):
        val_pred = model.predict(valid_model_input, verbose=1, batch_size=1024)   # [bs, 10]
        return auc(valid_label, val_pred)

evaluator = Evaluate()
history = model.fit(train_model_input, train_label,
                    batch_size=4096, epochs=20, verbose=1, callbacks=[evaluator])
# pred_ans = model.predict(test_model_input, batch_size=1024)

# np.save('test_ctr.npy', pred_ans)

Epoch 1/20


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


977/977 [==============================] - 10s 10ms/step
current auc: 0.799629, best auc: 0.799629

1465/1465 [==============================] - 113s 77ms/step - loss: 0.1301 - binary_crossentropy: 0.1297
Epoch 2/20
977/977 [==============================] - 10s 10ms/step
current auc: 0.801698, best auc: 0.801698

1465/1465 [==============================] - 113s 77ms/step - loss: 0.1240 - binary_crossentropy: 0.1236
Epoch 3/20
977/977 [==============================] - 10s 10ms/step
current auc: 0.801269, best auc: 0.801698

1465/1465 [==============================] - 111s 76ms/step - loss: 0.1234 - binary_crossentropy: 0.1229
Epoch 4/20
977/977 [==============================] - 10s 10ms/step
current auc: 0.801120, best auc: 0.801698

1465/1465 [==============================] - 112s 77ms/step - loss: 0.1230 - binary_crossentropy: 0.1225
Epoch 5/20
977/977 [==============================] - 10s 10ms/step
current auc: 0.800148, best auc: 0.801698

1465/1465 [=========================

KeyboardInterrupt: 

In [13]:
del model
gc.collect()

621377

In [22]:
# deepfm current auc: 0.798529, best auc: 0.798529
1+2 

3

In [37]:
model.load_weights("data/best_model_for_ctr.weights")

In [38]:
pred_ans1 = model.predict(test_model_input, batch_size=1024, verbose=1)
pred_ans1

977/977 [==============================] - 10s 10ms/step


array([[0.00572085],
       [0.06639403],
       [0.08771914],
       ...,
       [0.04968244],
       [0.01456261],
       [0.01481235]], dtype=float32)

In [25]:
pred_ans1 = model.predict(test_model_input, batch_size=1024, verbose=1)
pred_ans1 

977/977 [==============================] - 121s 124ms/step


array([[0.00349608],
       [0.07636404],
       [0.08461535],
       ...,
       [0.03903109],
       [0.01692522],
       [0.02210516]], dtype=float32)

In [34]:
pred_ans1 

array([[0.00346261],
       [0.15804213],
       [0.22009546],
       ...,
       [0.03756744],
       [0.01606992],
       [0.02256781]], dtype=float32)

In [ ]:
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': pred_ans1.reshape(-1), 
    })

result.to_csv("submission.csv", index=False) 

In [32]:
res1 = pd.read_csv("submission_fibinet7775.csv")
res2 = pd.read_csv("submission_xdeepfm7752.csv")

In [33]:
res1['probability'] = res1['probability'] * 0.55 + res2['probability'] * 0.45

In [34]:
res1.to_csv("submission.csv", index=False)